In [1]:
# !wget https://raw.githubusercontent.com/gururise/AlpacaDataCleaned/main/alpaca_data_cleaned.json

In [2]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
openai.api_type = 'azure'
openai.api_base = 'https://nous.openai.azure.com/'
openai.api_version = '2023-07-01-preview'
openai.api_key = ''
engine = 'nous'

In [4]:
def predict(prompt):
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [5]:
with open('alpaca_data_cleaned.json','r') as fopen:
    all_objs = json.load(fopen)

In [6]:
len(all_objs) // 2

25880

In [7]:
from depth import createConstraintsPrompt, createDeepenPrompt, createConcretizingPrompt, createReasoningPrompt
from breadth import createBreadthPrompt

In [8]:
!mkdir output
# !rm output/*

mkdir: cannot create directory ‘output’: File exists


In [9]:
def generate(instruction, index):
    filename = f'output/{index}.json'
    if os.path.exists(filename):
        return
    
    try:
        evol_prompts = []
        evol_prompts.append(createConstraintsPrompt(instruction))
        evol_prompts.append(createDeepenPrompt(instruction))
        evol_prompts.append(createConcretizingPrompt(instruction))
        evol_prompts.append(createReasoningPrompt(instruction))
        evol_prompts.append(createBreadthPrompt(instruction))

        selected_evol_prompt = random.choice(evol_prompts)
        ins = predict(selected_evol_prompt)
        answer = predict(ins + ', jawab dalam bahasa melayu')
        d = {"instruction":ins,"output":answer}
        with open(filename, 'w') as fopen:
            json.dump(d, fopen)
    except:
        pass

In [11]:
instructions = []
for cur_obj in all_objs:
    instructions.append(cur_obj['instruction'].strip() + '\r\n'+ cur_obj['input'].strip())

In [13]:
len(all_objs) - 1

51759

In [14]:
generate(instructions[-1], len(all_objs) - 1)

In [15]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range(len(all_objs) // 2, len(instructions), max_worker)):
    b = instructions[i: i + max_worker]
    b = [(ins, i + no) for no, ins in enumerate(b)]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f[0], f[1]): f for f in b}

        for future in as_completed(futures):
            future.result()

  1%|▏                                     | 54/8627 [08:51<24:30:41, 10.29s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


  1%|▍                                     | 86/8627 [13:27<19:00:56,  8.02s/it]

'content'


  1%|▍                                     | 90/8627 [13:51<16:19:38,  6.89s/it]

...

 99%|███████████████████████████████████▌| 8520/8627 [22:38:50<17:38,  9.89s/it]

'content'


 99%|███████████████████████████████████▋| 8563/8627 [22:45:22<09:26,  8.86s/it]

'content'


100%|████████████████████████████████████| 8627/8627 [22:54:46<00:00,  9.56s/it]
